In [2]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
from collections import Counter
import math
import numpy as np

In [3]:
df_main = pd.read_csv("../Dataset/raw/Indian_Food_Nutrition_Processed.csv")
df_recipes = pd.read_csv("../Dataset/raw/Cleaned_Indian_Food_Dataset.csv")
display(df_main)
display(df_recipes)

,Dish Name,Calories (kcal),Carbohydrates (g),Protein (g),Fats (g),Free Sugar (g),Fibre (g),Sodium (mg),Calcium (mg),Iron (mg),Vitamin C (mg),Folate (µg)
0,Hot tea (Garam Chai),16.14,2.58,0.39,0.53,2.58,0.00,3.12,14.20,0.02,0.50,1.80
1,Instant coffee,23.16,3.65,0.64,0.75,3.62,0.00,4.92,20.87,0.06,1.51,5.60
2,Espreso coffee,51.54,6.62,1.75,2.14,6.53,0.00,13.98,58.10,0.15,1.51,5.53
3,Iced tea,10.34,2.70,0.03,0.01,2.70,0.00,0.23,1.18,0.02,5.95,1.28
4,Raw mango drink (Aam panna),35.92,9.05,0.16,0.03,7.49,0.61,79.82,7.08,0.14,45.30,14.05
...,...,...,...,...,...,...,...,...,...,...,...,...
1009,Sweet and sour tomato pickle (Khatta meetha ta...,60.88,6.55,1.26,3.24,4.31,2.20,1281.94,15.18,0.54,NaN,NaN
1010,Jhatpat achar with carrot (Jhatpat achaar gaja...,91.21,6.32,1.98,6.55,3.04,5.08,2067.33,54.31,2.32,NaN,NaN
1011,Tomato chutney (Tamatar ki chutney),176.07,31.85,0.97,6.01,30.02,1.49,823.65,25.34,0.96,NaN,NaN
1012,Tomato ketchup,33.07,6.48,0.91,0.30,4.68,1.90,177.97,15.33,0.36,NaN,NaN


,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url,Ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...,12
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...,15
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seed...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...,12
...,...,...,...,...,...,...,...,...,...
5933,Saffron Paneer Peda Recipe,"2 teaspoons Rose water,1/2 teaspoon Ghee,2 cup...",20,Indian,To begin making the Saffron Paneer Peda recipe...,https://www.archanaskitchen.com/saffron-paneer...,"saffron strands generous,ghee,paneer,rose wate...",https://www.archanaskitchen.com/images/archana...,7
5934,Italian Arancini Rice Balls Recipe With Delici...,"4 sprig Basil leaves - finely chopped,2 cups W...",100,Italian Recipes,To begin making the Italian Arancini Rice Ball...,https://www.archanaskitchen.com/italian-aranci...,"salt,gorgonzola cheese cubes,wheat bread crumb...",https://www.archanaskitchen.com/images/archana...,14
5935,Quinoa Phirnee Recipe (Quinoa Milk Pudding),"3/4 cup Sugar,2 cup Milk - vegans can substitu...",35,Indian,"To begin making Quinoa Phirnee Recipe, place a...",https://www.archanaskitchen.com/quinoa-phirnee...,"ghee,milk,saffron strands,sugar,cardamom powde...",https://www.archanaskitchen.com/images/archana...,8
5936,Ullikadala Pulusu Recipe | Spring Onion Curry,"1 pinch Turmeric powder (Haldi),1/2 teaspoon M...",15,Andhra,To begin making Ullikadala Pulusu Recipe | Spr...,https://www.archanaskitchen.com/ullikadala-pul...,"salt,tamarind water,mustard seeds,asafoetida (...",https://www.archanaskitchen.com/images/archana...,13


In [4]:
# phrases to remove first (longer phrases first)
PHRASES_TO_REMOVE = [
    "how to make", "step by step", "home made", "homemade", "how to", "recipe"
]

# safe single words to remove (boilerplate/marketing)
SAFE_REMOVE = {
    "best","easy","simple","quick","tasty","delicious","instant",
    "healthy","authentic","traditional","classic","perfect", "style"
}

# measurements / units
UNITS = {
    "g","gram","grams","kg","ml","litre","liter","cup","cups",
    "tbsp","tsp","tablespoon","tablespoons","serving","servings",
    "slice","slices","piece","pieces"
}

# Aggressive words (do NOT enable by default)
AGGRESSIVE_REMOVE = {
    "masala","curry","spicy","fried","roasted","dry","gravy"
}

USE_AGGRESSIVE = False  # toggle this only after testing

# Composite scoring weights
WEIGHT_TOKENSET = 0.55
WEIGHT_WRATIO = 0.45

# Prefilter token overlap threshold (fraction of recipe tokens that must appear in candidate)
TOKEN_OVERLAP_MIN = 0.4   # tune: 0.4..0.6 is reasonable
TOP_K_CANDIDATES = 10     # consider top-K from process.extract for re-scoring

# -----------------------------
# Cleaning function
# -----------------------------
def clean_name_preserve(name: str) -> str:
    """Conservative cleaning:
       - remove long boilerplate phrases first
       - strip parentheses/units/numbers/punct
       - remove SAFE_REMOVE words, optionally AGGRESSIVE_REMOVE
       - return cleaned lowercased string
    """
    if not isinstance(name, str):
        return ""
    s = name.lower()

    # remove bracketed translations
    s = re.sub(r"\(.*?\)", " ", s)

    # remove long phrases (longest-first)
    for ph in sorted(PHRASES_TO_REMOVE, key=len, reverse=True):
        s = s.replace(ph, " ")

    # replace punctuation with space
    s = re.sub(r"[\|\,\/\\\-\_]+", " ", s)

    # remove measurements like "100 g", "2 cups"
    s = re.sub(
        r"\b\d+(\.\d+)?\s*(g|gram|grams|kg|ml|litre|liter|cup|cups|tbsp|tsp|tablespoon|tablespoons|serves|serving|servings)?\b",
        " ",
        s
    )
    s = re.sub(r"\b\d+\b", " ", s)

    # keep letters and spaces only
    s = re.sub(r"[^a-z\s]", " ", s)

    tokens = []
    for t in s.split():
        if t in SAFE_REMOVE:
            continue
        if t in UNITS:
            continue
        if USE_AGGRESSIVE and t in AGGRESSIVE_REMOVE:
            continue
        # keep all other tokens (including "spicy","masala","andhra", etc.)
        tokens.append(t)

    cleaned = " ".join(tokens)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

In [5]:

df_main["clean_name"] = df_main["Dish Name"].apply(clean_name_preserve)
df_recipes["clean_recipe"] = df_recipes["TranslatedRecipeName"].apply(clean_name_preserve)

# drop empties
df_main = df_main[df_main["clean_name"] != ""].copy()
df_recipes = df_recipes[df_recipes["clean_recipe"] != ""].copy()

print("After cleaning:")
print("Main:", df_main.shape)
print("Recipes:", df_recipes.shape)


After cleaning:
Main: (1014, 13)
Recipes: (5938, 10)


In [6]:
def token_overlap_ratio(a: str, b: str) -> float:
    sa = set(a.split())
    sb = set(b.split())
    if not sa:
        return 0.0
    return len(sa & sb) / len(sa)

In [7]:
matches = []

choices = df_main["clean_name"].tolist()

# -------------------------------
# FOOD TOKENS
# -------------------------------
FOOD_TOKENS = {
    "paneer","chicken","fish","egg","aloo","gobi","masala",
    "raita","paratha","kebab","biryani","dal","idli","dosa",
    "poha","upma","pulao","roti","sabzi","kheer","sambar",
    "rasam","soup","bhaji","sabji","tadka","fry","kurma",
    "matar","bhurji","makhani","butter","jeera","veg",
    "vegetable","gravy","tikka","bhuna","handi","do",
    "pyaza","methi","malai","kari","curry","korma","dum",
    "tawa","naan","chapati","phulka","kulcha","bhatura",
    "puri","poori","dry","fried","crispy","roasted",
    "stuffed","stuffing","spicy","tangy","creamy","rich",
    "grilled","idiyappam","uttapam","pakora","vada",
    "khichdi","kichdi","pongal","thepla","khakhra","misal",
    "usal","pav","pulav","baingan","brinjal","lauki",
    "bottle","gourd","tinda","palak","spinach","mushroom",
    "capsicum","shimla","mirch","corn","fenugreek","peas",
    "chole","chana","rajma","lobia","soy","soya","tofu",
    "cauliflower","garam","chilli","mirchi","pepper",
    "haldi","turmeric","ginger","garlic","tamarind","imli",
    "coconut","til","sesame","rice","lemon","curd",
    "yogurt","yoghurt","kadhai","kolhapuri","chettinad",
    "andhra","mangalore","hyderabadi","halwa","ladoo",
    "barfi","jalebi","payasam","sheera","modak","moong",
    "masoor","urad","toor","arhar","lentil"
}

# -------------------------------
# NEGATIVE PAIRS
# -------------------------------
NEGATIVE_TOKEN_PAIRS = {
    ("potato", "sweet"),
    ("salan", "soup"),
    ("rasam", "soup"),
    ("sambar", "soup"),
    ("pulao", "khichdi"),
    ("poriyal", "fry"),
    ("dal", "sambar"),
    ("dal", "rasam"),
    ("chutney", "gravy"),
    ("dosa", "roti"),
    ("idli", "dosa"),
    ("kheer", "payasam"),
    ("upma", "poha"),
    ("pulao", "curry"),
    ("singapore", "chinese")
}

def negative_pair_penalty(t1, t2):
    penalty = 0
    for a, b in NEGATIVE_TOKEN_PAIRS:
        if a in t1 and b in t2:
            penalty += 20
        if b in t1 and a in t2:
            penalty += 20
    return penalty

def food_token_score(t1, t2):
    return len(t1 & FOOD_TOKENS & t2)

# -------------------------------
# Map clean names → original names
# -------------------------------
choice_to_original = (
    df_main[["clean_name", "Dish Name"]]
    .drop_duplicates()
    .set_index("clean_name")["Dish Name"]
    .to_dict()
)

# -------------------------------
# MAIN MATCHING LOOP
# -------------------------------
for idx, row in df_recipes.iterrows():

    recipe_orig  = row["TranslatedRecipeName"]
    recipe_clean = row["clean_recipe"]

    # --- Step 1: Get top-K candidates
    raw_candidates = process.extract(
        recipe_clean,
        choices,
        scorer=fuzz.token_set_ratio,
        limit=TOP_K_CANDIDATES
    )

    scored_candidates = []

    for cand_name, token_set_score, cand_idx in raw_candidates:

        overlap = token_overlap_ratio(recipe_clean, cand_name)
        overlap_factor = overlap if overlap >= TOKEN_OVERLAP_MIN else overlap * 0.5

        wratio_score = fuzz.WRatio(recipe_clean, cand_name)

        rec_tokens = set(recipe_clean.split())
        cand_tokens = set(cand_name.split())

        ft_score = food_token_score(rec_tokens, cand_tokens)
        neg_penalty = negative_pair_penalty(rec_tokens, cand_tokens)

        composite = (
            WEIGHT_TOKENSET * token_set_score +
            WEIGHT_WRATIO  * wratio_score +
            12 * ft_score
        ) * (0.7 + 0.3 * overlap_factor) - neg_penalty

        scored_candidates.append(
            (cand_name, token_set_score, wratio_score, composite, overlap, ft_score, neg_penalty)
        )

    if scored_candidates:
        best = max(scored_candidates, key=lambda x: x[3])
        best_name, ts_score, wr_score, comp_score, overlap, ft_score, neg_penalty = best
    else:
        best_name, ts_score, wr_score, comp_score, overlap, ft_score, neg_penalty = ("",0,0,0,0,0,0)

    matches.append({
        "recipe_original": recipe_orig,
        "recipe_clean": recipe_clean,
        "best_match_clean": best_name,
        "matched_original_name": choice_to_original.get(best_name, ""),
        "token_set_score": round(ts_score, 2),
        "wratio_score": round(wr_score, 2),
        "food_token_score": ft_score,
        "negative_penalty": neg_penalty,
        "composite_score": round(comp_score, 2),
        "token_overlap": round(overlap, 2)
    })

# Final DataFrame with clean ordering
df_matches = pd.DataFrame(matches)[[
    "recipe_original",
    "recipe_clean",
    "best_match_clean",
    "matched_original_name",
    "token_set_score",
    "wratio_score",
    "food_token_score",
    "negative_penalty",
    "composite_score",
    "token_overlap"
]]


In [8]:
# Merge original Dish Name back
df_matches["matched_original_name"] = df_matches["best_match_clean"].map(choice_to_original)

# Quick filters: medium-confidence composite (you can tune)
df_medium = df_matches[(df_matches["composite_score"] >= 83) & (df_matches["token_set_score"] > 80)]

print("Total recipes:", len(df_recipes))
print("Medium composite matches:", df_medium.shape[0])
df_medium.head(120)



Total recipes: 5938
Medium composite matches: 744


,recipe_original,recipe_clean,best_match_clean,matched_original_name,token_set_score,wratio_score,food_token_score,negative_penalty,composite_score,token_overlap
3,Gongura Chicken Curry Recipe - Andhra Style Go...,gongura chicken curry andhra gongura chicken,chicken curry,Chicken curry,100.00,90.0,2,0,101.58,0.50
16,Hariyali Egg Curry Recipe In Coriander and Min...,hariyali egg curry in coriander and mint gravy,egg curry,Egg curry (Anda curry),100.00,90.0,2,0,88.13,0.25
17,Gourd Raita Recipe - Grated Bottle Gourd Yogur...,gourd raita grated bottle gourd yogurt salad,bottle gourd raita,Bottle gourd raita (Ghiya/Lauki ka raita),100.00,85.5,3,0,110.05,0.50
19,Chettinad Vegetable Casserole Recipe - Chettin...,chettinad vegetable casserole chettinad vegeta...,mixed vegetable pulao,Mixed vegetable pulao,83.33,85.5,2,0,92.06,0.50
37,Chettinad Style Chicken Roast Recipe,chettinad chicken roast,roast chicken,Roast chicken,100.00,85.5,1,0,94.93,0.67
...,...,...,...,...,...,...,...,...,...,...
935,Kollu Masala Dosa Recipe - High Protein Horse ...,kollu masala dosa high protein horse dosa,masala dosa,Masala dosa,100.00,90.0,2,0,89.62,0.33
939,Gujarati Vaghareli Mag Recipe - Whole Green Mo...,gujarati vaghareli mag whole green moong dal s...,dal moong,Dal moong,100.00,85.5,2,0,86.64,0.25
948,Chicken Sholay Kebab Recipe,chicken sholay kebab,chicken kebab,Chicken kebab,100.00,85.5,2,0,105.73,0.67
952,Tomato Onion Chutney Recipe - Tomato Onion Chu...,tomato onion chutney tomato onion chutney,onion chutney,Onion chutney (Pyaaz ki chutney/Vengaya chutney),100.00,90.0,0,0,85.95,0.67


In [9]:
# Define how close to the threshold counts as "risky"
COMP_LOW = 83
COMP_HIGH = 86     # upper limit for close-call rows
TSET_LOW = 80
TSET_HIGH = 85

df_risky = df_matches[
    (df_matches["composite_score"] >= COMP_LOW) &
    (df_matches["composite_score"] <= COMP_HIGH) &
    (df_matches["token_set_score"] >= TSET_LOW) &
    (df_matches["token_set_score"] <= TSET_HIGH)
].copy()

df_risky = df_risky.sort_values(
    ["composite_score", "token_set_score"],
    ascending=[True, True]
)

print("RISKY ROWS COUNT:", len(df_risky))
df_risky.head(40)


RISKY ROWS COUNT: 15


,recipe_original,recipe_clean,best_match_clean,matched_original_name,token_set_score,wratio_score,food_token_score,negative_penalty,composite_score,token_overlap
577,Tomato Noodle Soup Recipe,tomato noodle soup,cold tomato soup,Cold tomato soup,82.35,78.24,1,0,83.25,0.67
1814,Pakki Mutton Biryani Recipe,pakki mutton biryani,mutton biryani biriyani,Mutton biryani/biriyani,82.35,78.24,1,0,83.25,0.67
2389,Tomato Noodle Soup recipe,tomato noodle soup,cold tomato soup,Cold tomato soup,82.35,78.24,1,0,83.25,0.67
5304,Haryali Seekh Kebab Recipe,haryali seekh kebab,soya seekh kebab,Soya seekh kebab,81.48,80.00,1,0,83.53,0.67
1669,Aloo Bread Pakora Recipe,aloo bread pakora,bread pakora pakoda,Bread pakora/pakoda,83.33,79.17,1,0,84.11,0.67
3462,Stuffed Jackfruit Curry Recipe (Stuffed Jackfr...,stuffed jackfruit curry,jackfruit kofta curry,Jackfruit kofta curry (Kathal ka kofta curry),83.33,79.17,1,0,84.11,0.67
1458,Vegetarian Laksa Curry Recipe,vegetarian laksa curry,vegetarian egg kofta curry,Vegetarian egg kofta curry,84.21,80.00,1,0,84.88,0.67
3156,Asian Style Stuffed Omelette Recipe,asian stuffed omelette,stuffed egg omelette omlet,Stuffed egg omelette/omlet,84.21,80.00,1,0,84.88,0.67
4754,Pineapple Paneer Darts Recipe,pineapple paneer darts,paneer apple and pineapple salad,"Paneer, apple and pineapple salad",84.21,80.00,1,0,84.88,0.67
5348,Pumpkin Kofta Curry Recipe,pumpkin kofta curry,pea kofta curry,Pea kofta curry (Matar kofta curry),84.62,80.38,1,0,85.24,0.67


In [10]:
df_clean = df_medium.drop(df_risky.index).reset_index(drop=True)
df_risky.to_csv("../Dataset/raw/mismatched_rows_review.csv", index=False)


In [12]:
print(f"Starting the merge with {len(df_clean)} successfully matched recipes ")

df_rec_small = df_recipes[[
    "TranslatedRecipeName", "TranslatedIngredients", "TotalTimeInMins", "Cuisine",
    "TranslatedInstructions", "Cleaned-Ingredients"
]].rename(columns={"TranslatedRecipeName": "recipe_original"})

df_main_small = df_main[[
    "clean_name", "Dish Name", "Calories (kcal)", "Carbohydrates (g)", "Protein (g)",
    "Fats (g)", "Free Sugar (g)", "Fibre (g)", "Sodium (mg)", "Calcium (mg)",
    "Iron (mg)", "Vitamin C (mg)", "Folate (µg)"
]].rename(columns={"Dish Name": "final_food_name"})

df_merged_expanded = df_clean.merge(
    df_main_small,
    left_on="best_match_clean",
    right_on="clean_name",
    how="left"
)
# --- Step 5: ENFORCE 1-TO-1 MATCH (Final Clean Dataset of 729 rows) ---
# When a recipe matches multiple nutrition rows, keep only the one with the best score.
df_merged_expanded = df_merged_expanded.sort_values(
    'composite_score', ascending=False
)

df_final_1to1 = df_merged_expanded.drop_duplicates(
    subset=['recipe_original'], keep='first'
)
# 6. Final Merge to add remaining recipe details (Instructions, etc.)
df_final = df_final_1to1.merge(
    df_rec_small,
    on="recipe_original",
    how="left"
)

# 7. Final cleanup and reorder
df_final = df_final.drop(columns=["clean_name", "recipe_clean"])

# Define ordered columns
ordered_cols = [
    "recipe_original", "final_food_name", "TotalTimeInMins", "Cuisine",
    "TranslatedInstructions", "TranslatedIngredients", "Cleaned-Ingredients",
    "Calories (kcal)", "Carbohydrates (g)", "Protein (g)", "Fats (g)",
    "Free Sugar (g)", "Fibre (g)", "Sodium (mg)", "Calcium (mg)",
    "Iron (mg)", "Vitamin C (mg)", "Folate (µg)",
    "best_match_clean", "matched_original_name", "composite_score",
    "token_set_score", "wratio_score", "food_token_score",
    "negative_penalty", "token_overlap"
]

df_final = df_final[ordered_cols]

print(f"\n Final Unified Dataset Rows (Excluding Risky Data): {len(df_final)}")
display(df_final.head())

Starting the merge with 729 successfully matched recipes 

 Final Unified Dataset Rows (Excluding Risky Data): 729


,recipe_original,final_food_name,TotalTimeInMins,Cuisine,TranslatedInstructions,TranslatedIngredients,Cleaned-Ingredients,Calories (kcal),Carbohydrates (g),Protein (g),...,Vitamin C (mg),Folate (µg),best_match_clean,matched_original_name,composite_score,token_set_score,wratio_score,food_token_score,negative_penalty,token_overlap
0,Spinach Paneer Kofta Curry Recipe,Spinach paneer kofta curry (Palak paneer kofta...,100,North Indian Recipes,Method for KoftasHeat 1 teaspoon of oil in a p...,"5 cups Spinach Leaves (Palak) - chopped,Kasuri...","kasuri methi (dried fenugreek leaves) flavour,...",606.03,2.94,1.43,...,47.34,197.93,spinach paneer kofta curry,Spinach paneer kofta curry (Palak paneer kofta...,136.00,100.0,100.0,3,0,1.0
1,Urad Dal Vada Recipe (Non Deep Fried),Plain urad dal vada (Uzunne vada/Minapa garelu...,260,South Indian Recipes,To begin making Urad Dal Vada Recipe (Non Deep...,"1 Cup White Urad Dal (Split),2 inch Ginger,Sal...","salt,ginger,asafoetida (hing),virgin olive oil...",745.50,9.70,4.39,...,0.00,13.31,plain urad dal vada,Plain urad dal vada (Uzunne vada/Minapa garelu...,133.75,100.0,95.0,3,0,1.0
2,Chinese Chicken Sweet Corn Soup Recipe,Chicken sweet corn soup,80,Chinese,To begin making the Chinese Chicken Sweet Corn...,"1 inch Ginger - crushed,4 cloves Garlic - crus...","salt,ginger,corn flour,eggs,spring onion green...",28.32,1.63,14.48,...,2.02,13.45,chicken sweet corn soup,Chicken sweet corn soup,125.72,100.0,95.0,3,0,0.8
3,Mixed Vegetable Pulao Recipe,Mixed vegetable pulao,60,Indian,To begin making the Mixed Vegetable Pulao Reci...,"1 Carrots (Gajjar) - diced small,1 inch Cinnam...","green chillies,brown rice,salt,green beans (fr...",113.05,17.49,2.72,...,26.15,80.63,mixed vegetable pulao,Mixed vegetable pulao,124.00,100.0,100.0,2,0,1.0
4,Lemon Coriander Soup Recipe,Lemon coriander soup,30,Indo Chinese,To begin making the Lemon Coriander Soup Recip...,"Salt - to taste,1 tablespoon Lemon juice - fre...","salt,ginger,corn flour,coriander (dhania) leav...",108.06,10.32,2.27,...,8.60,12.47,lemon coriander soup,Lemon coriander soup,124.00,100.0,100.0,2,0,1.0


In [14]:
df_final.to_csv("../Dataset/processed/Final_unified_dataset.csv", index=False)

In [16]:
df_final.columns

Index(['recipe_original', 'final_food_name', 'TotalTimeInMins', 'Cuisine',
       'TranslatedInstructions', 'TranslatedIngredients',
       'Cleaned-Ingredients', 'Calories (kcal)', 'Carbohydrates (g)',
       'Protein (g)', 'Fats (g)', 'Free Sugar (g)', 'Fibre (g)', 'Sodium (mg)',
       'Calcium (mg)', 'Iron (mg)', 'Vitamin C (mg)', 'Folate (µg)',
       'best_match_clean', 'matched_original_name', 'composite_score',
       'token_set_score', 'wratio_score', 'food_token_score',
       'negative_penalty', 'token_overlap'],
      dtype='object')